In [1]:
import cv2
from os import chdir
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from V2I import video2image
import datetime
from moviepy.editor import *
import pandas as pd
import datetime
import pickle

# from keras.models import load_model
# my_vgg = load_model('finetuned_vgg.h5')

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

try:
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
except ImportError:
    pass

my_vgg = VGG16(weights='imagenet', include_top=False)

Using TensorFlow backend.


In [14]:
height = 128
width = 128
seq_len = 30
ms = 200

video_wd = '/Users/seogeurim/Desktop/FallDownDetection_Tobigs/TestData/'
model_saving_wd = '/Users/seogeurim/Desktop/FallDownDetection_Tobigs/Model1/Model_Saved_real/'

In [16]:
print(datetime.datetime.now())
data = np.zeros(128 * 128 * 3 * 30).reshape([1, 30, 128, 128, 3])
label = []
count = []
i = 0
for path, dir, files in os.walk(video_wd):
    if path == video_wd + 'NonViolence':
        idx = 0  # no _violence
    elif path == video_wd + 'Violence':
        idx = 1  # Violence
    for file in files:
        if file == '.DS_Store':
            print("DSSTORE=====")
        else:
            wd_file = path + '/' + file
            cnt, clips = video2image(wd=video_wd,
                                     video=wd_file,
                                     width=width,
                                     height=height,
                                     ms=ms,
                                     seq_len=seq_len,
                                     standard=50)
            data = np.concatenate([data, np.array(clips).reshape([1, 30, 128, 128, 3])], axis=0)
            label.append(idx)
            count.append(cnt)
            i += 1
            print('{} videos done.'.format(i))
label = np.array(label)
count = np.array(count)
count[count > seq_len] = seq_len
data = data[1:]
ch_to_idx = {'Violence': 1, 'NonViolence': 0}
n_videos = len(data)
n_class = len(ch_to_idx)
print(datetime.datetime.now())

2019-11-20 08:04:45.538472
DSSTORE=====
1 videos done.
2 videos done.
3 videos done.
4 videos done.
5 videos done.
6 videos done.
7 videos done.
8 videos done.
9 videos done.
10 videos done.
11 videos done.
12 videos done.
13 videos done.
14 videos done.
15 videos done.
16 videos done.
17 videos done.
18 videos done.
19 videos done.
20 videos done.
21 videos done.
22 videos done.
23 videos done.
24 videos done.
25 videos done.
26 videos done.
27 videos done.
28 videos done.
29 videos done.
30 videos done.
31 videos done.
32 videos done.
33 videos done.
34 videos done.
35 videos done.
36 videos done.
37 videos done.
38 videos done.
39 videos done.
40 videos done.
41 videos done.
42 videos done.
43 videos done.
44 videos done.
45 videos done.
46 videos done.
47 videos done.
48 videos done.
49 videos done.
50 videos done.
2019-11-20 08:05:14.658757


In [17]:
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [19]:
data_ftmap = np.zeros(30 * 4 * 4 * 512).reshape([1, 30, 4 * 4 * 512])
for i in range(len(data)):
    data_ftmap = np.concatenate([data_ftmap, my_vgg.predict(data[i]).reshape(1, seq_len, -1)], axis=0)
    if i % 10 == 0:
        print(i)

data_ftmap = data_ftmap[1:]

print(np.shape(data_ftmap))
print(np.shape(count))
print(np.shape(label))

0
10
20
30
40
(50, 30, 8192)
(50,)
(50,)


In [20]:
test_x = data_ftmap
test_y = label
test_count = count

print(np.shape(test_x))
print(np.shape(test_y))
print(np.shape(test_count))

(50, 30, 8192)
(50,)
(50,)


In [21]:
total_result = []
batch_size = 50
trying = 0  # 1st Model

tf.reset_default_graph()
test_graph = tf.Graph()
with tf.Session(graph=test_graph) as sess:
    loader = tf.train.import_meta_graph(model_saving_wd + str(trying) + '_Model.ckpt.meta')
    loader.restore(sess, model_saving_wd + str(trying) + '_Model.ckpt')

    LSTM_x = test_graph.get_tensor_by_name(name='LSTM_x:0')
    LSTM_y = test_graph.get_tensor_by_name(name='LSTM_y:0')
    LSTM_count = test_graph.get_tensor_by_name(name='LSTM_count:0')
    pred = test_graph.get_tensor_by_name(name='pred:0')
    acc = test_graph.get_tensor_by_name(name='acc:0')

    for i in range(int(len(test_x) / batch_size) + 1):
        if i == int(n_videos / batch_size):  # 배치 다돌고 나머지 처리할떄
            batch_test_x = test_x[i * batch_size:]
            batch_label = test_y[i * batch_size:]
            batch_count = test_count[i * batch_size:]

        else:
            batch_test_x = test_x[i * batch_size:(i + 1) * batch_size]
            batch_label = test_y[i * batch_size:(i + 1) * batch_size]
            batch_count = test_count[i * batch_size:(i + 1) * batch_size]

        print('test accuracy : ', sess.run(acc, feed_dict={LSTM_x: batch_test_x,
                                                           LSTM_y: batch_label,
                                                           LSTM_count: batch_count}))
        prediction = sess.run(pred, feed_dict={LSTM_x: batch_test_x,
                                               LSTM_count: batch_count})
        print(batch_label)
        print(batch_count)

        total_result += list(prediction)

print(pd.crosstab(np.array(total_result), test_y))

test accuracy :  0.56
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
[17 16 16 16 11 12 16 14 15 14 18 19 17 14 16 17  9 14 17 12 16 14 13 17
 15 21 24  9 28 13 20 15 26 17 26  6 24 30 21 23  7 20 18 18 17 19  8 26
 25  7]
test accuracy :  nan
[]
[]
col_0   0   1
row_0        
0.0    15  10
1.0    10  15
